# lab_01 - Czy na pewno już potrzebujesz narzędzi BIG DATA?

# 1. Przetwarzanie rozproszone - wady i zalety.

Po narzędzia do przetwarzania dużych zbiorów danych sięgamy zazwyczaj wtedy, kiedy ich przetwarzanie lokalnie staje się niemożliwe lub problematyczne. Zazwyczaj wtedy, gdy ilość przetwarzanych danych nie jest możliwa do "upchnięcia" w dostępnej pamięci RAM (lub pamięci kart GPU) i szukamy sposobu na skalowanie architektury poziomo.
Jednak czy zrobiliśmy wystarczająco dużo, aby zoptymalizować wykorzystanie tej pamięci? 
Przetwarzanie rozproszone, które jest sercem przetwarzania danych o wolumenie big data, ma swoje zalety, ale również i wady.

**Zalety przetwarzania rozproszonego:**
* możliwość rozłożenia pracy na większą ilość węzłów, których sumaryczna wydajność (procesor) oraz ilość pamięci może być wielokrotnie większa niż sprzętu dostępnego lokalnie,
* możliwość dość łatwego skalowania klastra w razie potrzeby,
* awaria pojedynczego węzła nie musi zakończyć się niepowodzeniem całego procesu przetwarzania danych,

**Wady przetwarzania rozproszonego:**
* wymaga dostępu do klastra (konfiguracja, dostępy),
* przy nieodpowiednio dobranej konfiguracji do wielkości zbioru danych przetwarzanie może zająć więcej czasu niż lokalnie:
  * w zależności od tego gdzie znajdują się dane - czas ich przesłania do klastra i propagacji na poszczególne węzły może być długi,
  * nieodpowiednie dobranie wielkości partycji (w sensie fragmentów zbioru do przetworzenia) do parametrów klastra może spowodować, że węzły przez większośc czasu będą oczekiwały na zakończenie zadań zależnych zamiast faktycznie coś liczyć. Poprawne dobranie parametrów wymaga doświadczenia, ale czasem wielu prób przed wdrożeniem produkcyjnym.
* koszt.

# 2. Optymalizacja. Co możemy zrobić?

## 2.1 Optymalizacja wykorzystania pamięci.

Każdy punkt danych, który jest wczytany do pamięci RAM zajmuje jej część w zależności od typu, jaki został mu przydzielony. W zależności od języka programowania oraz wybranej struktury danych te wielkości mogą się bardzo różnić. Przykłady optymalizacji zaprezentowane zostaną w języku Python i z użyciem najpopularniejszych bibliotekach do przetwarzania danych czyli numpy oraz pandas.

Proces doboru bardziej optymalnego typu danych można rozszerzyć o zmianę zakresu (dziedziny) tych danych, który nazywa się kwantyzacją (ang. quantization) i jest obecnie powszechnie stosowany m.in. do zmniejszania rozmiarów modeli LLM.

Optymalizacja pamięci możliwa jest również na poziomie pamięci dyskowej, gdzie powstały bardziej zoptymalizowane formaty przechowywania danych niż te najbardziej popularne wśród osób pracujących na co dzień w obszarze data science. Są to między innymi formaty:
* Parquet,
* ORC,
* AVRO.

Temat formatów danych zostanie omówiony w późniejszym czasie.

## 2.2 Multiprocessing lokalnie.

Nie które języki programowania natywnie wykorzystują wszystkie dostępne rdzenie i nie musimy się zbyt często martwić, aby jako programista wysokopoziomowy optymalizować samodzielnie ten kod na kod wielowątkowy lub wieloprocesowy. CPython, który jest najbardziej popularną implementacją interpretera języka Python, posiada dość poważne ograniczenie w postaci blokady GIL (czytaj więcej: [tu](https://realpython.com/python-gil/), [tu](https://wiki.python.org/moin/GlobalInterpreterLock) oraz [tu](https://bulldogjob.pl/readme/python-dazy-do-usuniecia-gil-i-zwiekszenia-wspolbieznosci)), która dotyczy wielu popularnych bibliotek.

## 2.3 Python - biblioteka Numba.

Jest to biblioteka dedykowana do optymalizacji ciężkich obliczeń numerycznych poprzez możliwość kompilacji odpowiednio napisanego kodu w języku Python do kodu maszynowego. Numba współpracuje z biblioteką Dask oraz systememSpark, które zostaną zaprezentowane w późniejszym czasie. Możliwe jest również wykorzystanie bilioteki CUDA w celu wykonywania obliczeń na kartach graficznych firmy NVIDIA.

# 3. Optymalizowanie danych w bibliotece pandas.

## 3.1 Rozgrzewka.

Pakiety niezbędne do wykonania kodu z bieżącego labu:
* pandas
* numpy
* jupyter-lab
* fastparquet
* filesplit

In [ ]:
# Optymalizację możemy zacząć już na etapie procesu wczytywania danych do pandas DataFrame.
import pandas as pd
import numpy as np
from datetime import datetime

# wczytywanie danych z pliku "na raz" - przy małych plikach optymalne rozwiązanie
df = pd.read_csv('zamowienia.csv', header=0, sep=';')
display(df.head())
# poniższa funkcja zwróci nam między innymi typy danych dla każdej kolumny i SZACUNKOWĄ wielkość pamięci, którą zajmuje
# nie jest to jednak wielkość dokładna
df.info()

In [ ]:
# aby sprawdzić ilość pamięci zajmowaną przez ramkę (lub serię) danych, skorzystamy z funkcji memory_usage
df.memory_usage()

Informacja została podana dla każdej kolumny (również indeksu) wyrażona w bajtach. Zwrócony typ danych? Pandas series. Możemy więc to zsumować.

In [ ]:
sum(df.memory_usage())

Ta liczba bajtów odpowiada informacji podanej po wywołaniu `df.info()`, ale domyślna wartość parametru `deep=False` powoduje, że nie są to ponownie informacje dokładne. Sprawdźmy więc ile to jest dokładnie.

In [ ]:
sum(df.memory_usage(deep=True))

Widać teraz, że faktycznie jest to wielkość kilkukrotnie większa.

Na potrzeby naszych eksperymentów wykorzystamy funkcję, która będzie nam tłumaczyła bajty na coś bardziej przyjaznego (za: https://stackoverflow.com/questions/1094841/get-a-human-readable-version-of-a-file-size).

In [ ]:
def sizeof_fmt(num, suffix="B"):
    for unit in ("", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"):
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"

In [ ]:
sizeof_fmt(sum(df.memory_usage(deep=True)))

## 3.2 Optymalizacja wczytywania plików w bibliotece pandas.

Plik źródłowy jest mały, więc trudno będzie miarodajnie zmierzyć różnice pomiędzy różnymi sposobami jego wczytywania. Sztucznie zwielokrotnimy więc dane we wczytanej ramce danych i zapiszemy do nowego pliku.

In [ ]:
# zwiększamy ramkę 50 000 razy - uwaga z wartością tego parametru w zależności od ilości dostępnej pamięci RAM
new_df = pd.concat([df.sample(frac=1) for n in range(50_000)])

In [ ]:
new_df.info()

In [ ]:
sizeof_fmt(sum(new_df.memory_usage(deep=True)))

In [ ]:
new_df.to_csv('zamowienia_expanded.csv', header=True, index=False)

In [ ]:
start = datetime.now()
new_df = pd.read_csv('zamowienia_expanded.csv', header=0)
print(f"Czas wczytywania case 1: {datetime.now() - start} sekund")

In [ ]:
# aby nie dodawać każdorazowo linii kodu z pomiarem czasu opakujemy tę część w dekorator, który można wielokrotnie reużywać

def count_time(func):
    def wrapper(*args, **kwargs):
        start = datetime.now()
        func(*args, **kwargs)
        print(f"Czas wczytywania {func.__name__}: {datetime.now() - start} sekund")
        return func(*args, **kwargs)
    return wrapper

In [ ]:
# Dekoratorów można używać w postaci adnotacji poprzedzającej definicję funkcji, którą następnie musimy jeszcze wywołać.
# Poniższe dwie funkcje wczytują plik csv na dwa sposoby, pierwsza wczytuje plik "na raz", a druga dzieląc go na części
# składające się z ilości linii przekazanych przez parametr chunksize. Każdy wczytany fragment to oddzielna ramka danych,
# którą możemy scalić lub przetwarzać oddzielnie.

@count_time
def read_file_1():
    return pd.read_csv('zamowienia_expanded.csv', header=0)
    
@count_time
def read_file_2():
    chunks = pd.read_csv('zamowienia_expanded.csv', header=0, chunksize=4_000_000)
    return pd.concat(chunks)

In [ ]:
df1 = read_file_1()
df2 = read_file_2()

> Porównując jedynie czas wykonania, niewielką przewagę będzie posiadała metoda wczytująca plik "na raz", jednak jeżeli popatrzymy na utylizację pamięci RAM w trakcie obu procesów to w zależności od systemu operacyjnego jej wykorzystanie może się różnić. W systemie Windows przy wykorzystaniu metody wczytującej plik we fragmentach można dostrzec spadki wykorzystania pamięci RAM w dość równych odstępach czasu. Będzie się to zbiegało z wczytywaniem kolejnych chunków. To powoduje, że maksymalny peak utylizacji pamięci RAM będzie niższy w przypadku wczytywania z podziałem na części. Im większy plik, tym ta różnica będzie wzrastać.

In [ ]:
sizeof_fmt(sum(df1.memory_usage(deep=True))), sizeof_fmt(sum(df2.memory_usage(deep=True)))

Warto też zwrócić uwagę na różnicę w rozmiarze pliku csv vs. rozmiar w pamięci RAM po wczytaniu do pandas DataFrame z domyślnymi typami danych.

**Inne formaty plików**

In [ ]:
# format parquet
import fastparquet

df1.to_parquet('zamowienia_expanded.parquet', engine='fastparquet')

In [ ]:
@count_time
def read_parquet_1():
    df = pd.read_parquet('zamowienia_expanded.parquet', engine='fastparquet')
    return df

In [ ]:
df3 = read_parquet_1()

In [ ]:
sizeof_fmt(sum(df3.memory_usage(deep=True)))

**Multiprocessing**

Ten kod należy uruchomić poza Jupyter Notebookiem, gdyż nie jest on obsługiwany dla tego przypadku. Pamiętaj o dodaniu zdefiniowanej wcześniej funkcji `count_time`.

In [ ]:
from itertools import repeat
import pandas as pd
from datetime import datetime
from filesplit.split import Split
from multiprocessing import Pool
import os


def apply_args_and_kwargs(func, args, kwargs):
    return func(*args, **kwargs)


def starmap_with_kwargs(pool, func, args_iter, kwargs_iter):
    args_for_starmap = zip(repeat(func), args_iter, kwargs_iter)
    return pool.starmap(apply_args_and_kwargs, args_for_starmap)


def split_file(filepath, chunksize, destination):
    split = Split(filepath, destination)
    split.bylinecount(linecount=chunksize, includeheader=True)


@count_time
def load_files(directory):

    files = [[f"{directory}/{f}"] for f in os.listdir(directory) if f.endswith(".csv")]

    kwargs_list = [
        {
            'on_bad_lines': "skip",
        }
        for n in range(len(files))
    ]

    pool = Pool(processes=5)
    args_iter = files

    results = starmap_with_kwargs(pool, pd.read_csv, args_iter, kwargs_list)
    results = pd.concat(results)

    return results


if __name__ == '__main__':
    split_file('zamowienia_expanded.csv', 8_000_000, 'data')
    df4 = load_files('data')
    df4.info()

## 3.3 Optymalizacja wykorzystania pamięci RAM ramek biblioteki pandas

Każda kolumna danych z pliku wczytanego do ramki pandas otrzymuje swój typ, który wynika z zawartości danych w tej kolumnie. Przydzielanie tych typów może być automatyczne (domyślnie), ale można również wskazać pożądany typ lub zmienić go już po wczytaniu danych. Automatyczne przydzielanie typów bywa czasami bardzo nieoptymalne pod kątem wykorzystania pamięci RAM i może w pewnych przypadkach uniemożliwić przetwarzanie zbioru (błędy out of memory), który po optymalizacji tych typów, może na danej maszynie jednak być przetworzony.

In [ ]:
# dla przypomnienia zerknijmy na typy danych ustawione automatycznie
df1.info() # lub df1.dtypes

Mamy 3 kolumny typu 'object' (typ str) oraz po jednej typu int64 oraz float64. Pamiętajmy tutaj, że biblioteka pandas wykorzystuje struktury danych z biblioteki numpy (która jest wrapperem do stosownego kodu napisanego w języku C) do przechowywania danych. Mamy więc do dyspozycji znacznie więcej typów niż natywnie dostępne standardowo w Pythonie. Więcej tutaj: https://numpy.org/doc/stable/user/basics.types.html

In [ ]:
# poznanie zakresu danych powinno pomóc w ocenie czy dobrany typ danych numerycznych jest optymalny
df1.describe()

In [ ]:
# zmiana domyślnej precyzji formatu wyświetlania danych
pd.options.display.float_format = '{:.5f}'.format
df1.describe()

In [ ]:
for column in df1.columns:
    print(f'{column}: {sizeof_fmt(df1[column].memory_usage(deep=True))}')

In [ ]:
sizeof_fmt(df1['idZamowienia'].astype(np.int16).memory_usage(deep=True))

In [ ]:
sizeof_fmt(df1['Kraj'].astype('category').memory_usage(deep=True))

In [ ]:
sizeof_fmt(df1['Sprzedawca'].astype('category').memory_usage(deep=True))

In [ ]:
sizeof_fmt(pd.to_datetime(df1['Data zamowienia']).memory_usage(deep=True))

In [ ]:
# tworzymy pistą ramkę danych, aby przechować w niej dane w nowym, bardziej optymalnym formacie
df2 = pd.DataFrame()

In [ ]:
# zmieniamy format niektórych kolumn
df2['Kraj'] = df1['Kraj'].astype('category')
df2['Sprzedawca'] = df1['Sprzedawca'].astype('category')
df2['Data zamowienia'] = pd.to_datetime(df1['Data zamowienia'])
df2['idZamowienia'] = df1['idZamowienia'].astype(np.int16)
df2['Utarg'] = df1['Utarg']

In [ ]:
sizeof_fmt(sum(df2.memory_usage(deep=True)))

In [ ]:
# możemy również spróbować wykonać downcasting dla kolumn numerycznych wykorzystując wbudowaną funkcję biblioteki panda to_numeric
utarg_downcast = pd.to_numeric(df2["Utarg"], downcast='float')
sizeof_fmt(utarg_downcast.memory_usage(deep=True)), utarg_downcast.dtype

In [ ]:
# ostatecznie uzyskamy
df2['Utarg'] =  pd.to_numeric(df1["Utarg"], downcast='float')
sizeof_fmt(sum(df2.memory_usage(deep=True)))

In [ ]:
# i dla każdej kolumny oddzielnie
for column in df2.columns:
    print(f'{column}: {sizeof_fmt(df2[column].memory_usage(deep=True))}')

In [ ]:
df2.info()

In [ ]:
df2.describe()

#### Porównanie czasów wykonania dla oryginalnej ramki oraz ramki zoptymalizowanej 

In [ ]:
start = datetime.now()
display(df1.groupby(['Sprzedawca']).agg({'Utarg': ['mean']}))
print(f'Czas: {datetime.now() - start}')

In [ ]:
start = datetime.now()
display(df2.groupby(['Sprzedawca']).agg({'Utarg': ['mean']}))
print(f'Czas: {datetime.now() - start}')

## Zadania

> Zbiór danych do wykonania zadań: https://huggingface.co/datasets/vargr/private_instagram/tree/refs%2Fconvert%2Fparquet/default/train
>
> **UWAGA!**  
> W zależności od ilości pamięci RAM pobierz tyle plików, aby możliwe było wczytanie danych do pamięci RAM.
> Spróbuj dobrać tyle danych, aby maksymalnie wykorzystać pamięć operacyjną.
> Możesz również spróbować dobrać więcej danych niż zmieści się w pamięci operacyjnej w celu wywołania błędu biblioteki pandas 

**Zadanie 1**  
Wczytaj pliki danych i scal je w jedną ramkę DataFrame.
Wykonaj analizę typów danych podobnie jak w przykładach.
Zmierz wielkość pamięci RAM ramki z domyślnymi typami danych.

**Zadanie 2**  
Dobierz bardziej optymalne typy danych i ponownie zmierz wielkość zajmowanej pamięci RAM.
Porównaj obie wielkości na wykresie (wybierz pasujący typ wykresu).

**Zadanie 3**  
Wykonaj 3 wybrane operacje (grupowanie + agregacja, filtrowanie, itp.) na całej ramce i zmierz czas wykonania na danych oryginalnych i zoptymalizowanych.
Wyświetl te czasy.

**Zadanie 4**  
Zapisz ramkę jako plik csv, z nagłówkami kolumn, bez indeksu.
Sprawdź jaka jest różnica w wielkości pliku csv i sumy wielkości plików w formacie parquet (w eksploratorze, nie trzeba tego robić z poziomu kodu).

**Zadanie 5**  
Zmierz czas wczytywania danych z pliku csv dla 3 przypadków:
* cały plik na raz,
* cały plik ze wskazaniem parametru `chunksize` (możesz poeksperymentować z wielkością tego parametru),
* z użyciem multiprocessingu zaprezentowanego w przykładzie (wcześniej podziel plik na kilka mniejszych), wskazując ilość procesów jako `ilość_rdzeni - 2` oraz drugi przypadek `(ilosc_rdzeni - 2) * 2`.


**Zadanie 6** (z gwiazdką, nie jest obowiązkowe, ale pouczające)  

Wczytaj każdy plik podzielony w zadaniu 5 do oddzielnej ramki danych. 
Dla każdej ramki policz sumę na kolumnie `likes`, a następnie policz sumę tych sum. Tę część zadania wykonaj sekwencyjnie.
Teraz wykorzystując multiprocessing (i przykłady z labu) wykonaj to samo zadanie zrównoleglając je. Zmierz czas obu przypadków i go wyświetl.
